In [113]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn

In [114]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [115]:
dataset = pd.read_csv('cereal.csv',delimiter=',')
x = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

In [116]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
x=ct.fit_transform(x)
from sklearn.preprocessing import StandardScaler
scx=StandardScaler()
scy=StandardScaler()
x[1:]=scx.fit_transform(x[1:])
y=y.reshape(-1,1)
y=scy.fit_transform(y)
y=y.reshape(-1)

In [117]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state =42)

In [118]:
x_train=np.vstack(x_train).astype(float)
x_train=torch.FloatTensor(x_train).to(device)
x_test=np.vstack(x_test).astype(float)
x_test=torch.FloatTensor(x_test).to(device)
y_train=np.vstack(y_train).astype(float)
y_train=torch.FloatTensor(y_train).to(device)
y_test=np.vstack(y_test).astype(float)
y_test=torch.FloatTensor(y_test).to(device)

In [119]:
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1=nn.Linear(len(x[0]),50,device=device)
        self.layer_2=nn.Linear(50,50,device=device)
        self.layer_3=nn.Linear(50,1,device=device)
        self.act=nn.ReLU()
    def forward(self,x):
        return self.act(self.layer_3(self.act(self.layer_2(self.act(self.layer_1(x))))))
    
model=Regressor().to(device)

In [120]:
loss_fn=nn.MSELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.001)

In [121]:
def accuracy_fn(y_true, y_pred):
    y_true=y_true.reshape(-1,1)
    y_pred=y_pred.reshape(-1,1)
    y_true,y_pred=scy.inverse_transform(y_true),scy.inverse_transform(y_pred)
    correct=0
    for i in range(len(y_pred)):
        correct+=abs(y_pred[i]-y_true[i])/y_true[i]
    acc = correct / len(y_pred)*100
    return acc

In [122]:
torch.manual_seed(42)
epochs=10001

for epoch in range(epochs):
    model.train()
    y_logits=model(x_train).squeeze()
    loss=loss_fn(y_logits,y_train.squeeze())
    y_pred=y_logits.detach().cpu().numpy()
    y_train1=y_train.detach().cpu().numpy()
    err=accuracy_fn(y_train1,y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.inference_mode():
        test_logits=model(x_test).squeeze()
        test_loss=loss_fn(test_logits,y_test.squeeze())
        test_pred=test_logits.detach().cpu().numpy()
        y_test1=y_test.detach().cpu().numpy()
        test_err=accuracy_fn(y_test1,test_pred)
        if(epoch%1000==0):
            print(f"Epoch: {epoch} | Loss: {loss:.5f}, Err: {err[0]:.2f}% | Test Loss: {test_loss:.5f}, Test Err: {test_err[0]:.2f}%")

Epoch: 0 | Loss: 1.04810, Err: 28.01% | Test Loss: 0.81080, Test Err: 29.50%
Epoch: 1000 | Loss: 1.04810, Err: 28.01% | Test Loss: 0.81080, Test Err: 29.50%
Epoch: 2000 | Loss: 1.04810, Err: 28.01% | Test Loss: 0.81080, Test Err: 29.50%
Epoch: 3000 | Loss: 1.04810, Err: 28.01% | Test Loss: 0.81080, Test Err: 29.50%
Epoch: 4000 | Loss: 1.04810, Err: 28.01% | Test Loss: 0.81080, Test Err: 29.50%
Epoch: 5000 | Loss: 1.04810, Err: 28.01% | Test Loss: 0.81080, Test Err: 29.50%
Epoch: 6000 | Loss: 1.04810, Err: 28.01% | Test Loss: 0.81080, Test Err: 29.50%
Epoch: 7000 | Loss: 1.04810, Err: 28.01% | Test Loss: 0.81080, Test Err: 29.50%
Epoch: 8000 | Loss: 1.04810, Err: 28.01% | Test Loss: 0.81080, Test Err: 29.50%
Epoch: 9000 | Loss: 1.04810, Err: 28.01% | Test Loss: 0.81080, Test Err: 29.50%
Epoch: 10000 | Loss: 1.04810, Err: 28.01% | Test Loss: 0.81080, Test Err: 29.50%
